In [1]:
!pip install transformers
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 5.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hgn4i57g
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-hgn4i57g
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369398 sha256=c99b2d2eea0f139d9000f12f3a4223139fd26db0878a946892d42439525b79aa
  Sto

In [2]:
import pandas as pd
import urllib.request
from multiprocessing.pool import ThreadPool
from PIL import Image
import math
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
import torch
import tensorflow as tf
from transformers import AutoProcessor, TFCLIPModel
import clip

In [3]:
!wget https://unsplash.com/data/lite/latest

--2023-04-17 20:19:55--  https://unsplash.com/data/lite/latest
Resolving unsplash.com (unsplash.com)... 151.101.1.181, 151.101.65.181, 151.101.129.181, ...
Connecting to unsplash.com (unsplash.com)|151.101.1.181|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unsplash-datasets.s3.amazonaws.com/lite/latest/unsplash-research-dataset-lite-latest.zip [following]
--2023-04-17 20:19:56--  https://unsplash-datasets.s3.amazonaws.com/lite/latest/unsplash-research-dataset-lite-latest.zip
Resolving unsplash-datasets.s3.amazonaws.com (unsplash-datasets.s3.amazonaws.com)... 3.5.10.204, 52.217.230.129, 3.5.8.111, ...
Connecting to unsplash-datasets.s3.amazonaws.com (unsplash-datasets.s3.amazonaws.com)|3.5.10.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 632351052 (603M) [application/zip]
Saving to: ‘latest’

latest              100%[===================>] 603.06M  12.8MB/s    in 60s     

2023-04-17 20:20:57 (10.0 MB/s) - ‘latest’

In [4]:
!mkdir unsplash-dataset
!mkdir unsplash-dataset/lite/
!mkdir unsplash-dataset/lite/photos
!mkdir unsplash-dataset/lite/features


In [5]:
!unzip "/content/latest" -d "/content/unsplash-dataset/lite"

Archive:  /content/latest
  inflating: /content/unsplash-dataset/lite/collections.tsv000  
  inflating: /content/unsplash-dataset/lite/__MACOSX/._collections.tsv000  
  inflating: /content/unsplash-dataset/lite/colors.tsv000  
  inflating: /content/unsplash-dataset/lite/__MACOSX/._colors.tsv000  
  inflating: /content/unsplash-dataset/lite/conversions.tsv000  
  inflating: /content/unsplash-dataset/lite/__MACOSX/._conversions.tsv000  
  inflating: /content/unsplash-dataset/lite/DOCS.md  
  inflating: /content/unsplash-dataset/lite/keywords.tsv000  
  inflating: /content/unsplash-dataset/lite/__MACOSX/._keywords.tsv000  
  inflating: /content/unsplash-dataset/lite/photos.tsv000  
  inflating: /content/unsplash-dataset/lite/__MACOSX/._photos.tsv000  
  inflating: /content/unsplash-dataset/lite/README.md  
  inflating: /content/unsplash-dataset/lite/TERMS.md  


In [6]:
from pathlib import Path
dataset_version = "lite" 
unsplash_dataset_path = Path("unsplash-dataset") / dataset_version

In [7]:
# Read the photos table
photos = pd.read_csv(unsplash_dataset_path / "photos.tsv000", sep='\t', header=0)

# Extract the IDs and the URLs of the photos
photo_urls = photos[['photo_id', 'photo_image_url']].values.tolist()

# Print some statistics
print(f'Photos in the dataset: {len(photo_urls)}')

Photos in the dataset: 25000


In [8]:
# Path where the photos will be downloaded
photos_donwload_path = unsplash_dataset_path / "photos"

# Function that downloads a single photo
def download_photo(photo):
    # Get the ID of the photo
    photo_id = photo[0]

    # Get the URL of the photo (setting the width to 640 pixels)
    photo_url = photo[1] + "?w=640"

    # Path where the photo will be stored
    photo_path = photos_donwload_path / (photo_id + ".jpg")

    # Only download a photo if it doesn't exist
    if not photo_path.exists():
        try:
            urllib.request.urlretrieve(photo_url, photo_path)
        except:
            # Catch the exception if the download fails for some reason
            print(f"Cannot download {photo_url}")
            pass

In [9]:
# Create the thread pool
threads_count = 128
pool = ThreadPool(threads_count)

# Start the download
pool.map(download_photo, photo_urls)

# Display some statistics
display(f'Photos downloaded: {len(photos)}')

Cannot download https://images.unsplash.com/photo-1498144846853-60ca2d43853b?w=640
Cannot download https://images.unsplash.company%20by%20Alessandro%20Desantis%20-%20Downloaded%20from%20500px_jpg.jpg?w=640
Cannot download https://images.unsplash.com/photo-1578411246981-e0394f597159?w=640
Cannot download https://images.unsplash.com_TheBeach.jpg?w=640
Cannot download https://images.unsplash.com/photo-1578166671353-7978081a6f9c?w=640
Cannot download https://images.unsplash.com/photo-1583307709837-2d0e3a82be15?w=640
Cannot download https://images.unsplash.com-grass-sun.jpg?w=640
Cannot download https://images.unsplash.com/photo-1573486145949-182147241fa6?w=640


'Photos downloaded: 25000'

In [10]:
dataset_version = "lite"  # Use "lite" or "full"
photos_path = Path("unsplash-dataset") / dataset_version / "photos"

# List all JPGs in the folder
photos_files = list(photos_path.glob("*.jpg"))

# Print some statistics
print(f"Photos found: {len(photos_files)}")

Photos found: 24992


In [11]:
import clip
import torch
from PIL import Image

# Load the open CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Function that computes the feature vectors for a batch of images
def compute_clip_features(photos_batch):
    # Load all the photos from the files
    photos = [Image.open(photo_file) for photo_file in photos_batch]
    
    # Preprocess all photos
    photos_preprocessed = torch.stack([preprocess(photo) for photo in photos]).to(device)

    with torch.no_grad():
        # Encode the photos batch to compute the feature vectors and normalize them
        photos_features = model.encode_image(photos_preprocessed)
        photos_features /= photos_features.norm(dim=-1, keepdim=True)

    # Transfer the feature vectors back to the CPU and convert to numpy
    return photos_features.cpu().numpy()

100%|███████████████████████████████████████| 338M/338M [00:25<00:00, 14.0MiB/s]


In [12]:
import math
import numpy as np
import pandas as pd

# Define the batch size so that it fits on your GPU. You can also do the processing on the CPU, but it will be slower.
batch_size = 32

# Path where the feature vectors will be stored
features_path = Path("unsplash-dataset") / dataset_version / "features"

# Compute how many batches are needed
batches = math.ceil(len(photos_files) / batch_size)

# Process each batch
for i in range(batches):
    print(f"Processing batch {i+1}/{batches}")

    batch_ids_path = features_path / f"{i:010d}.csv"
    batch_features_path = features_path / f"{i:010d}.npy"
    
    # Only do the processing if the batch wasn't processed yet
    if not batch_features_path.exists():
        try:
            # Select the photos for the current batch
            batch_files = photos_files[i*batch_size : (i+1)*batch_size]

            # Compute the features and save to a numpy file
            batch_features = compute_clip_features(batch_files)
            np.save(batch_features_path, batch_features)

            # Save the photo IDs to a CSV file
            photo_ids = [photo_file.name.split(".")[0] for photo_file in batch_files]
            photo_ids_data = pd.DataFrame(photo_ids, columns=['photo_id'])
            photo_ids_data.to_csv(batch_ids_path, index=False)
        except:
            # Catch problems with the processing to make the process more robust
            print(f'Problem with batch {i}')

Processing batch 1/781
Processing batch 2/781
Processing batch 3/781
Processing batch 4/781
Processing batch 5/781
Processing batch 6/781
Processing batch 7/781
Processing batch 8/781
Processing batch 9/781
Processing batch 10/781
Processing batch 11/781
Processing batch 12/781
Processing batch 13/781
Processing batch 14/781
Processing batch 15/781
Processing batch 16/781
Processing batch 17/781
Processing batch 18/781
Processing batch 19/781
Processing batch 20/781
Processing batch 21/781
Processing batch 22/781
Processing batch 23/781
Processing batch 24/781
Processing batch 25/781
Processing batch 26/781
Processing batch 27/781
Processing batch 28/781
Processing batch 29/781
Processing batch 30/781
Processing batch 31/781
Processing batch 32/781
Processing batch 33/781
Processing batch 34/781
Processing batch 35/781
Processing batch 36/781
Processing batch 37/781
Processing batch 38/781
Processing batch 39/781
Processing batch 40/781
Processing batch 41/781
Processing batch 42/781
P

In [13]:
# Load all numpy files
features_list = [np.load(features_file) for features_file in sorted(features_path.glob("*.npy"))]

# Concatenate the features and store in a merged file
features = np.concatenate(features_list)
np.save(features_path / "features.npy", features)

# Load all the photo IDs
photo_ids = pd.concat([pd.read_csv(ids_file) for ids_file in sorted(features_path.glob("*.csv"))])
photo_ids.to_csv(features_path / "photo_ids.csv", index=False)

In [14]:
model = TFCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

All model checkpoint layers were used when initializing TFCLIPModel.

All the layers of TFCLIPModel were initialized from the model checkpoint at openai/clip-vit-base-patch32.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCLIPModel for predictions without further training.


In [15]:
photo_ids = pd.read_csv('/content/unsplash-dataset/lite/features/photo_ids.csv')
photo_ids = list(photo_ids['photo_id'])
photo = np.load('/content/unsplash-dataset/lite/features/features.npy')
print(f'Photos Loaded: {len(photo_ids)}')
photo_features = tf.convert_to_tensor(
    photo
)

Photos Loaded: 24992


In [16]:
def encode_search_query(query):
  encoded_text = processor(query, return_tensors = 'tf')
  encoded_text = model.get_text_features(**encoded_text)
  encoded_text /= tf.norm(encoded_text, axis = -1,  keepdims = True)
  return encoded_text

In [17]:
def find_best_match(text_features, photo_features, photo_ids, count = 3):
  similarities = tf.squeeze(photo_features @ tf.cast(tf.transpose(text_features), tf.float16))
  best_photo_idx = tf.argsort(-similarities)
  return [photo_ids[i] for i in best_photo_idx[:count]]

In [18]:
from IPython.display import Image
from IPython.core.display import HTML

def display_photo(photo_id):
  # Get the URL of the photo resized to have a width of 320px
  photo_image_url = f"https://unsplash.com/photos/{photo_id}/download?w=320"

  # Display the photo
  display(Image(url=photo_image_url))

  # Display the attribution text
  display(HTML(f'Photo on <a target="_blank" href="https://unsplash.com/photos/{photo_id}">Unsplash</a> '))
  print()

In [19]:
def search_unslash(search_query, photo_features, photo_ids, results_count=3):
  # Encode the search query
  text_features = encode_search_query(search_query)

  # Find the best matches
  best_photo_ids = find_best_match(text_features, photo_features, photo_ids, results_count)

  # Display the best photos
  for photo_id in best_photo_ids:
    display_photo(photo_id)

In [20]:
search_query = "people in beach"

search_unslash(search_query, photo_features, photo_ids, 3)